In [6]:
import torch
import torch.nn as nn
import chess

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
class kcolbvonc(nn.Module):
    def __init__(self, ic, oc, ks, st, pd, active: bool = True, ip:bool = True):
        super(kcolbvonc, self).__init__()
        
        self.ip = ip
        self.tive = active
        self.kcolb = self.kcolbkcolb(ic, oc, ks, st, pd)
        
    def kcolbkcolb(self, ic, oc, ks, st, pd):
        
        if not self.tive:
            
            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
            )
        
        else:

            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
                nn.LeakyReLU(inplace = self.ip)
            )
            
        return kcolbconv
    
    def forward(self, idk) -> torch.Tensor:
        
        return self.kcolb(idk)

In [9]:
class laud(nn.Module):
    def __init__(self, ic, oc, ks, st, pd, nnn: int, jump: bool = True):
        super(laud, self).__init__()
        
        self.ic = ic
        self.oc = oc
        self.ks = ks
        self.st = st
        self.pd = pd
        self.j = jump
        self.nnn = nnn
        self.nc: int = 0
        
        self.reyal = self.laudreyal()
        self.matcher = kcolbvonc(ic, oc, 1, 1, 0, active = False)
        self.smatcher = nn.AdaptiveMaxPool2d((8, 8))
        
    def forward(self, idk):
        
        logitnotgood = idk
        
        for layer in self.reyal:
            logitnotgood = layer(logitnotgood)
        
        if self.j and (self.ic != self.oc):
            
            idk = self.matcher(idk)
            logitnotgood = self.smatcher(logitnotgood)
            
            logitgood = idk + logitnotgood
                        
        elif self.j and (self.ic == self.oc):

            logitnotgood = self.smatcher(logitnotgood)
            
            logitgood = idk + logitnotgood
        
        else:
            
            logitgood = logitnotgood
            print('using a residual block builder to build a conv block? are you outta your mind...h/e heres your tensor')

        return logitgood

    def laudreyal(self):
            
        reyal = nn.ModuleList()
        self.nc = self.ic
        
        for nnnn in range(self.nnn):
            
            ncc = self.ic * (nnnn + 2)
            
            rayal = kcolbvonc(self.nc, ncc, self.ks, self.st, self.pd)

            reyal.append(rayal)
            self.nc = ncc


        rayalo = kcolbvonc(self.nc, self.oc, self.ks, self.st, self.pd)

        reyal.append(rayalo)
        
        return reyal

In [10]:
dummyo = torch.rand((2, 12, 8, 8), device = device)
dummyoo = torch.rand((2, 120, 8, 8), device = device)

In [11]:
class cheslermod(nn.Module):
    def __init__(self): 
        super(cheslermod, self).__init__()
        
        self.alin = nn.Sequential(
            laud(ic = 120, oc = 256, ks = 2, st = 1, pd = 1, nnn = 2), 
            laud(ic = 256, oc = 128, ks = 2, st = 1, pd = 1, nnn = 2), 
        )
        
        self.stein = nn.Sequential(
            laud(ic = 12, oc = 128, ks = 2, st = 1, pd = 1, nnn = 2),
        )
        
        self.after = nn.Sequential(
            laud(ic = 128, oc = 32, ks = 2, st = 1, pd = 1, nnn = 2), 
            nn.Flatten(), 
            self.lfter(2048, 1024, True), 
            self.lfter(1024, 512, True), 
            self.lfter(512, 120, False)
        )
        
    def forward(self, oo, o):
        
        i = self.alin(oo)
        ii = self.stein(o)
        
        z = i + ii
        
        zz = self.after(z)
        
        return zz
    
    @staticmethod
    def lfter(ic, oc, nfl):
        
        if nfl:
            
            l = nn.Sequential(
                nn.Linear(ic, oc), 
                nn.BatchNorm1d(oc), 
                nn.LeakyReLU(inplace = True)
            )        
        
        else:

            l = nn.Sequential(
                nn.Linear(ic, oc),
                nn.Softmax(dim = -1)
            )
            
        return l

In [12]:
cheslerdom = cheslermod().to(device)

In [13]:
a = cheslerdom(dummyoo, dummyo)
print(a.shape)
print(sum(a[-1]))
print(a[0])
print(torch.argmax(a[0]))

torch.Size([2, 120])
tensor(1., device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0068, 0.0047, 0.0132, 0.0052, 0.0036, 0.0116, 0.0068, 0.0121, 0.0109,
        0.0132, 0.0108, 0.0080, 0.0154, 0.0109, 0.0086, 0.0080, 0.0070, 0.0069,
        0.0057, 0.0081, 0.0047, 0.0087, 0.0109, 0.0167, 0.0091, 0.0084, 0.0107,
        0.0061, 0.0069, 0.0062, 0.0090, 0.0058, 0.0036, 0.0057, 0.0051, 0.0093,
        0.0103, 0.0088, 0.0031, 0.0051, 0.0049, 0.0030, 0.0075, 0.0059, 0.0140,
        0.0069, 0.0093, 0.0098, 0.0064, 0.0045, 0.0109, 0.0078, 0.0078, 0.0033,
        0.0060, 0.0079, 0.0078, 0.0055, 0.0024, 0.0137, 0.0105, 0.0068, 0.0044,
        0.0099, 0.0059, 0.0080, 0.0093, 0.0040, 0.0060, 0.0068, 0.0057, 0.0103,
        0.0046, 0.0060, 0.0104, 0.0188, 0.0057, 0.0088, 0.0086, 0.0178, 0.0057,
        0.0074, 0.0156, 0.0158, 0.0036, 0.0067, 0.0071, 0.0144, 0.0066, 0.0158,
        0.0070, 0.0110, 0.0079, 0.0172, 0.0092, 0.0085, 0.0146, 0.0120, 0.0058,
        0.0080, 0.0070, 0.0167, 0.0077, 0.0088,

In [14]:
def t(tensor):
    return torch.tensor(tensor, device = device, dtype = torch.float16)

In [15]:
def LMs(board: chess.Board):

    legal_moves = list(board.legal_moves)
    legal_moves_san = [board.san(moves) for moves in legal_moves]
    legal_moves_uci = [board.uci(moves) for moves in legal_moves]

    return legal_moves_uci, legal_moves_san

In [16]:
def ecode_board(board: chess.Board):

    board_enc = torch.zeros((12, 8, 8), device = device)

    oh_label = {
        'P': 0,
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'p': 6,
        'r': 7,
        'n': 8,
        'b': 9,
        'q': 10,
        'k': 11
    }

    for square in chess.SQUARES:

        piece = board.piece_at(square)

        if piece:

            board_count = oh_label[str(piece)]
            row_count, col_count = torch.tensor(divmod(square, 8), device = device)
            col_count = torch.abs(col_count - 7)

            board_enc[board_count, row_count, col_count] = 1

    return board_enc

In [17]:
def ecode_moves(moves: list):

    state = torch.zeros((120, 8, 8), device = device)
    # col_dict = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    col_dict = {'a': 7, 'b': 6, 'c': 5, 'd': 4, 'e': 3, 'f': 2, 'g': 1, 'h': 0}
    for ind, move in enumerate(moves[:120]):

        o_col = torch.tensor(col_dict[move[0]], device = device)
        # o_row = torch.abs(torch.tensor(int(move[1]) - 1, device = device) - 7)
        o_row = torch.tensor(int(move[1]) -1, device = device)
        e_col = torch.tensor(col_dict[move[2]], device = device)
        e_row = torch.tensor(int(move[3]) - 1, device = device)

        state[ind, o_row, o_col] = 1

        if move[-1] == 'q':
            state[ind, e_row, e_col] = 3
        else:
            state[ind, e_row, e_col] = 2

    return state

In [53]:
def pan_sys(pred_ind, board_state: chess.Board, legal_uci, legal_san, v, captures):


    if pred_ind >= len(legal_uci): 
        
        total_lm_penalty = torch.tensor(0, dtype = torch.float32, device = device)
        total_action_penalty = torch.tensor(0, dtype = torch.float32, device = device)
        move = None
        # v_val = None
    
    else:
        move = legal_uci[pred_ind]
        actionn = legal_san[pred_ind]
        action = actionn.replace('x', '').replace('+', '').replace('q', '').replace('#', '').replace('=', '')
    
        file = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        s_piece = ['R', 'N', 'B']
        capture = 'x'
    
    
        if 2 < len(action) < 6 and action[0] in file:
            action = f'p{action[1:3]}'
        if len(actionn) == 2:
            action = f'p{actionn}'
        elif len(actionn) >= 6:
            action = f'p{actionn[2:4]}'
        elif (capture in actionn) and (actionn[0] in s_piece) and (actionn[1] in file):
            action = actionn[:1] + actionn[2:]
    
        col_dict = {'a': t(0), 'b': t(1), 'c': t(2), 'd': t(3), 'e': t(4), 'f': t(5), 'g': t(6), 'h': t(7)}
    
        penalty_ = {'p': t(2), 'n': t(4), 'b': t(4), 'r': t(7), 'q': t(10), 'k': t(float('inf'))}
    
        total_lm_penalty = torch.tensor(0.5, dtype = torch.float32, device = device)
        total_action_penalty = torch.tensor(0., dtype = torch.float32, device = device)
    
        if '#' in actionn:
            total_action_penalty = total_action_penalty + 3
    
        base_penalty = 0.5
    
        if (capture not in actionn) and captures:
            total_action_penalty = total_action_penalty - 0
            # v_val = None
    
        elif  (capture not in actionn) and (not captures):
            total_action_penalty = torch.tensor(0, dtype = torch.float32, device = device)
    
        else:
            try:
                cap_loc = action[1:3]
                square = (torch.tensor(int(cap_loc[1]), device = device) - 1) * 8 + int(col_dict[cap_loc[0]])
                I_piece = action[0].lower()
                o_piece = str(board_state.piece_at(square.item())).lower()
                refI = penalty_[I_piece]
                refO = penalty_[o_piece]
    
                if refI < refO:
                    mul = (refO - refI) / 2
                    penalty_score = base_penalty * refO * mul
                elif refI > refO:
                    mul = (refI - refO)
                    penalty_score = base_penalty * (refO / mul)
                elif refI == refO:
                    penalty_score = base_penalty
                else:
                    penalty_score = 0
    
                total_action_penalty  = total_action_penalty + penalty_score
    
            except Exception as e:
                total_action_penalty = 'error'
    
            # v_val = bellman(v)
    
    return total_action_penalty + total_lm_penalty, move #, v_val

In [54]:
def pg_loss(prob, r):
    if not prob == 0 or not prob.is_nan():
        loss = -torch.sum(torch.log(prob) * r)
    else:
        loss = None

    return loss

In [48]:
from torch.distributions import Categorical

In [63]:
class renair: 
    def __init__(self, learning_rate): 
        
        self.dom = cheslermod().to(device)
        self.chesleropt = torch.optim.Adam(lr = learning_rate, params = self.dom.parameters())
        self.accurws = [] 
        self.accuprobs = [] 
        self.gmrws = []
        
    def update(self, stacked_moves, stacked_states, uci, san, cap): 
        
        stigol = self.dom(stacked_moves, stacked_states) 
        probs = Categorical(stigol) 
        inxs = probs.sample() 
        sampleprobs = [stigol[d, inxs[d]] for d in range(len(inxs))] 
        
        for inx, prob, state in zip(inxs, sampleprobs, stacked_states): 
            
            rw, move = pan_sys(inx, stacked_states, uci, san, 0, cap) 
            self.accurws.append(rw) 
            self.accuprobs.append(prob) 
        
    def train(self, γ): 
        
        for rw in self.accurws: 
            self.gmrws.append((rw * γ)) 
        
        self.gmrws, self.accuprobs, self.accurws = t(self.gmrws), t(self.accuprobs), t(self.accurws) 
        
        self.accuprobs = torch.log(self.accuprobs) 
        
        loss = -(self.accuprobs * self.gmrws).mean() 
        
        self.chesleropt.zero_grad() 
        loss.backward() 
        self.chesleropt.step() 
    
renairt = renair(0.001) 

In [64]:
from tqdm import tqdm

In [65]:
def train(epos):
    epos = range(epos) 
    for epo in tqdm(epos): 
        
        step = 0 
        envir = chess.Board() 
        
        acc_move_c = 0
        
        acc_moves = [] 
        acc_boards = [] 
        
        while True: 
            
            mate = envir.is_checkmate() 
            draw = envir.is_stalemate() or envir.is_insufficient_material() 
            fin = mate or draw 
            
            if fin: 
                break

            legal_ucii, legal_sann = LMs(envir)

            captures = False

            for legal_san in legal_sann:
                if 'x' in legal_san:
                    captures = True

            board = ecode_board(envir) 
            moves = ecode_moves(legal_ucii) 
            
            if not acc_move_c == 2:
                
                acc_moves.append(moves) 
                acc_boards.append(board) 
                acc_move_c += 1
            
            else:
                
                acc_moves.append(moves)
                acc_boards.append(board)
                acc_move_c = 0
                
                acc_moves = torch.stack(acc_moves) 
                acc_boards = torch.stack(acc_boards) 
                
                renairt.train(acc_moves, acc_boards, legal_ucii, legal_sann, captures)
                
                break

            rand_uci = [envir.uci(moves) for moves in envir.legal_moves]
            rand_move = rand_uci[torch.randint(0, len(rand_uci), (1, 1))]
            move_rand = chess.Move.from_uci(rand_move)
            envir.push(move_rand)

In [66]:
train(1)

100%|██████████| 1/1 [00:00<00:00, 24.37it/s]
